In [30]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [60]:
# Resize all to same shape and convert them to tensor
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
    ]),
}

# Create Dataloaders and datasets for train and test set
data_dir = 'https://drive.google.com/drive/folders/1jJ7TTKbpxXbXJEhy4mJ32Dd0G9W6JAfa?usp=share_link'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


device(type='cpu')

In [61]:
class_names

['bear', 'butterfly', 'camel', 'chimp', 'duck', 'elephant']

In [63]:
# Loading Resnet18 model
model_conv = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [64]:
# total Batch shape
# Each batch has 4 images, each of size 3*128*128
inputs.shape

torch.Size([128, 3, 224, 224])

In [65]:
# Output of resnet for a batch
model_conv(inputs).shape

torch.Size([128, 1000])

In [66]:
# Architecture of resnet18
list(model_conv.children())

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
   (1): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pad

In [69]:
# Exclude the final layer
feature_ext = torch.nn.Sequential(*list(model_conv.children())[:-1])

# Output of a batch after passing to feature_ext
feature_ext(inputs).reshape((128,-1)).shape

torch.Size([128, 512])

In [70]:
for param in feature_ext.parameters():
    param.requires_grad = False

In [71]:
batch_size = 128

In [72]:
# Formulate train and test dataset by getting feature vector of 512 size
d = 512
X_train = np.empty((0,d))
Y_train = np.empty((0,1))
X_test = np.empty((0,d))
Y_test = np.empty((0,1))

for inputs, labels in dataloaders['train']:
  features = feature_ext(inputs).reshape(inputs.shape[0],-1)
  X_train = np.append(X_train,features.numpy(),axis = 0)
  Y_train = np.append(Y_train,labels.numpy().reshape(-1,1),axis = 0)

for inputs, labels in dataloaders['test']:
  features = feature_ext(inputs).reshape(inputs.shape[0],-1)
  X_test = np.append(X_test,features.numpy(),axis = 0)
  Y_test = np.append(Y_test,labels.numpy().reshape(-1,1),axis = 0)

In [73]:
inputs.shape

torch.Size([120, 3, 224, 224])

In [74]:
print(f"X_train shape -- {X_train.shape}")
print(f"Y_train shape -- {Y_train.shape}")
print(f"X_test shape -- {X_test.shape}")
print(f"X_test shape -- {Y_test.shape}")

X_train shape -- (532, 512)
Y_train shape -- (532, 1)
X_test shape -- (120, 512)
X_test shape -- (120, 1)


In [84]:
# K-NN classifier
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, Y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [85]:
# Assigns a vector to a dominating class among K-Nearest Neighbours
predictions = neigh.predict(X_test)

In [86]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, predictions, target_names=class_names))

              precision    recall  f1-score   support

        bear       0.90      0.95      0.93        20
   butterfly       0.95      1.00      0.98        20
       camel       0.94      0.85      0.89        20
       chimp       0.95      0.95      0.95        20
        duck       1.00      0.90      0.95        20
    elephant       0.91      1.00      0.95        20

    accuracy                           0.94       120
   macro avg       0.94      0.94      0.94       120
weighted avg       0.94      0.94      0.94       120



In [87]:
# Accuracy as fraction of correctly classfied test images
sum(predictions.reshape(-1,1)==Y_test).astype(int)/Y_test.shape[0]

array([0.94166667])